In [1]:
from pybaseball import statcast_batter_expected_stats
from pybaseball import pitching_stats
from pybaseball import statcast_batter
from pybaseball import playerid_lookup
import pandas as pd
from scipy.stats import zscore
from pybaseball import cache
from unidecode import unidecode

## Batters by wBOA ZScore

In [2]:
from pybaseball import batting_stats
# retrieve data on only players who have 50+ plate appearances this year
full_batting_2018 = batting_stats(2018, 50)
full_batting_2019 = batting_stats(2019, qual=150)
full_batting_2021 = batting_stats(2021, qual=150)
full_batting_2022 = batting_stats(2022, qual=150)
full_batting_2023 = batting_stats(2023, qual=150)

# Concatenate the DataFrames vertically
combined_batting_data = pd.concat([full_batting_2018, full_batting_2019,full_batting_2021, full_batting_2022, full_batting_2023], ignore_index=False)
full_batting_2022.head()

full_batting_2018

,IDfg


In [3]:
from pybaseball import statcast_batter_expected_stats

batting_18 = statcast_batter_expected_stats(2018, 150)
batting_19 = statcast_batter_expected_stats(2019, 150)
batting_21 = statcast_batter_expected_stats(2021, 150)
batting_22 = statcast_batter_expected_stats(2022, 150)
batting_23 = statcast_batter_expected_stats(2023, 150)

batting_data = pd.concat([batting_18, batting_19, batting_21, batting_22, batting_23], ignore_index=True)
batting_data

,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff
0,Lindor,Francisco,596019,2018,745,560,0.277,0.289,-0.012,0.519,0.512,0.007,0.368,0.375,-0.007
1,Turner,Trea,607208,2018,740,534,0.271,0.273,-0.002,0.416,0.415,0.001,0.331,0.335,-0.004
2,Machado,Manny,592518,2018,709,533,0.297,0.284,0.013,0.538,0.512,0.026,0.377,0.369,0.008
3,Hernández,César,514917,2018,708,454,0.253,0.242,0.011,0.362,0.352,0.010,0.320,0.316,0.004
4,Merrifield,Whit,593160,2018,707,526,0.304,0.281,0.023,0.438,0.428,0.010,0.349,0.343,0.006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,Wong,Kolten,543939,2023,216,150,0.165,0.210,-0.045,0.227,0.284,-0.057,0.216,0.260,-0.044
1634,d'Arnaud,Travis,518595,2023,214,150,0.250,0.263,-0.013,0.444,0.471,-0.027,0.323,0.344,-0.021
1635,Arroyo,Christian,624414,2023,206,153,0.241,0.231,0.010,0.369,0.341,0.028,0.274,0.269,0.005
1636,Espinal,Santiago,669289,2023,199,154,0.218,0.239,-0.021,0.296,0.319,-0.023,0.265,0.284,-0.019


In [4]:
# Combine 'first_name' and 'last_name' columns to create the 'name' column
batting_data['name'] = batting_data.apply(lambda row: f"{row['first_name']} {row['last_name']}", axis=1)

# Drop the 'first_name' and 'last_name' columns
batting_data.drop(['first_name', 'last_name'], axis=1, inplace=True)

# Reorder columns so 'name' is the first column
new_column_order = ['name'] + [col for col in batting_data.columns if col != 'name']
batting_data = batting_data[new_column_order]

# Iterate through each row and clean the 'name' column using unidecode
for index, row in batting_data.iterrows():
    cleaned_name = unidecode(row['name'])
    batting_data.at[index, 'name'] = cleaned_name


batting_data

,name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff
0,Francisco Lindor,596019,2018,745,560,0.277,0.289,-0.012,0.519,0.512,0.007,0.368,0.375,-0.007
1,Trea Turner,607208,2018,740,534,0.271,0.273,-0.002,0.416,0.415,0.001,0.331,0.335,-0.004
2,Manny Machado,592518,2018,709,533,0.297,0.284,0.013,0.538,0.512,0.026,0.377,0.369,0.008
3,Cesar Hernandez,514917,2018,708,454,0.253,0.242,0.011,0.362,0.352,0.010,0.320,0.316,0.004
4,Whit Merrifield,593160,2018,707,526,0.304,0.281,0.023,0.438,0.428,0.010,0.349,0.343,0.006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,Kolten Wong,543939,2023,216,150,0.165,0.210,-0.045,0.227,0.284,-0.057,0.216,0.260,-0.044
1634,Travis d'Arnaud,518595,2023,214,150,0.250,0.263,-0.013,0.444,0.471,-0.027,0.323,0.344,-0.021
1635,Christian Arroyo,624414,2023,206,153,0.241,0.231,0.010,0.369,0.341,0.028,0.274,0.269,0.005
1636,Santiago Espinal,669289,2023,199,154,0.218,0.239,-0.021,0.296,0.319,-0.023,0.265,0.284,-0.019


In [5]:
# Filter the DataFrame for the relevant years (2019, 2021, 2022)
relevant_years = [2018, 2019, 2021, 2022]
filtered_batting_data = batting_data[batting_data['year'].isin(relevant_years)]


woba_2023 = batting_23[['player_id', 'woba']].rename(columns={'woba': 'woba_2023'})
batting_grouped_data = filtered_batting_data .groupby(['player_id', 'name']).mean().reset_index().rename(columns={'woba': 'avg_woba'})

# Merge player names with the result DataFrame
# Merge player names and calculated results
players_batting_output = pd.merge(
    batting_grouped_data,
    woba_2023,
    on=['player_id']
)

In [6]:
column_order = [
    col for col in players_batting_output.columns if col not in ['avg_woba', 'woba_2023']
] + ['avg_woba', 'woba_2023']

players_batting_output = players_batting_output[column_order]

players_batting_output

,player_id,name,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,est_woba,est_woba_minus_woba_diff,avg_woba,woba_2023
0,408234,Miguel Cabrera,2020.666667,502.666667,351.333333,0.26400,0.2460,0.01800,0.36700,0.395667,-0.028667,0.30700,-0.007667,0.299333,0.290
1,444482,David Peralta,2020.000000,516.250000,365.500000,0.26950,0.2540,0.01550,0.44850,0.398750,0.049750,0.31500,0.018750,0.333750,0.301
2,453568,Charlie Blackmon,2020.000000,622.250000,457.500000,0.28475,0.2795,0.00525,0.47700,0.449000,0.028000,0.34325,0.008000,0.351250,0.357
3,455117,Martin Maldonado,2020.000000,395.750000,252.500000,0.19900,0.1945,0.00450,0.34525,0.334250,0.011000,0.26800,0.003250,0.271250,0.252
4,456781,Donovan Solano,2020.666667,292.000000,213.666667,0.29800,0.2860,0.01200,0.41500,0.425000,-0.010000,0.33100,-0.001333,0.329667,0.340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,681082,Bryson Stott,2022.000000,466.000000,340.000000,0.23400,0.2370,-0.00300,0.35800,0.340000,0.018000,0.28400,0.005000,0.289000,0.338
243,682928,CJ Abrams,2022.000000,302.000000,238.000000,0.24600,0.2410,0.00500,0.32400,0.316000,0.008000,0.26600,0.001000,0.267000,0.309
244,682985,Riley Greene,2022.000000,418.000000,259.000000,0.25300,0.2460,0.00700,0.36200,0.388000,-0.026000,0.31600,-0.013000,0.303000,0.354
245,683734,Andrew Vaughn,2021.500000,512.000000,370.000000,0.25300,0.2490,0.00400,0.41250,0.424500,-0.012000,0.32550,-0.009000,0.316500,0.322


In [7]:
avg_woba = players_batting_output['avg_woba']
woba_2023 = players_batting_output['woba_2023']

z_scores_avg_woba = zscore(avg_woba)
z_score_woba_2023 = zscore(woba_2023)

# Add new columns with Z-scores to the DataFrame
players_batting_output['z_scores_avg_woba'] = z_scores_avg_woba

In [8]:
# # Calculate z-scores for avg_wOBA and wOBA_2023

players_batting_output['zscore_difference'] = z_score_woba_2023 - z_scores_avg_woba 

players_batting_output

,player_id,name,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,est_woba,est_woba_minus_woba_diff,avg_woba,woba_2023,z_scores_avg_woba,zscore_difference
0,408234,Miguel Cabrera,2020.666667,502.666667,351.333333,0.26400,0.2460,0.01800,0.36700,0.395667,-0.028667,0.30700,-0.007667,0.299333,0.290,-0.932775,0.004742
1,444482,David Peralta,2020.000000,516.250000,365.500000,0.26950,0.2540,0.01550,0.44850,0.398750,0.049750,0.31500,0.018750,0.333750,0.301,0.179349,-0.804794
2,453568,Charlie Blackmon,2020.000000,622.250000,457.500000,0.28475,0.2795,0.00525,0.47700,0.449000,0.028000,0.34325,0.008000,0.351250,0.357,0.744836,0.170167
3,455117,Martin Maldonado,2020.000000,395.750000,252.500000,0.19900,0.1945,0.00450,0.34525,0.334250,0.011000,0.26800,0.003250,0.271250,0.252,-1.840247,-0.133090
4,456781,Donovan Solano,2020.666667,292.000000,213.666667,0.29800,0.2860,0.01200,0.41500,0.425000,-0.010000,0.33100,-0.001333,0.329667,0.340,0.047402,0.399965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,681082,Bryson Stott,2022.000000,466.000000,340.000000,0.23400,0.2370,-0.00300,0.35800,0.340000,0.018000,0.28400,0.005000,0.289000,0.338,-1.266682,1.659032
243,682928,CJ Abrams,2022.000000,302.000000,238.000000,0.24600,0.2410,0.00500,0.32400,0.316000,0.008000,0.26600,0.001000,0.267000,0.309,-1.977579,1.572198
244,682985,Riley Greene,2022.000000,418.000000,259.000000,0.25300,0.2460,0.00700,0.36200,0.388000,-0.026000,0.31600,-0.013000,0.303000,0.354,-0.814292,1.646771
245,683734,Andrew Vaughn,2021.500000,512.000000,370.000000,0.25300,0.2490,0.00400,0.41250,0.424500,-0.012000,0.32550,-0.009000,0.316500,0.322,-0.378060,0.330283


In [9]:
# Filter the DataFrame for the relevant years (2019, 2021, 2022)
relevant_years = [2018, 2019, 2021, 2022]
filtered_batting_data = batting_data[batting_data['year'].isin(relevant_years)]


slug_2023 = batting_23[['player_id', 'slg']].rename(columns={'slg': 'slg_2023'})
power_grouped_data = filtered_batting_data.groupby(['player_id', 'name']).mean().reset_index().rename(columns={'slg': 'avg_slg'})

# Merge player names with the result DataFrame
# Merge player names and calculated results
players_power_output = pd.merge(
    power_grouped_data,
    slug_2023,
    on=['player_id']
)

players_power_output

,player_id,name,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,avg_slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,slg_2023
0,408234,Miguel Cabrera,2020.666667,502.666667,351.333333,0.26400,0.2460,0.01800,0.36700,0.395667,-0.028667,0.299333,0.30700,-0.007667,0.332
1,444482,David Peralta,2020.000000,516.250000,365.500000,0.26950,0.2540,0.01550,0.44850,0.398750,0.049750,0.333750,0.31500,0.018750,0.399
2,453568,Charlie Blackmon,2020.000000,622.250000,457.500000,0.28475,0.2795,0.00525,0.47700,0.449000,0.028000,0.351250,0.34325,0.008000,0.460
3,455117,Martin Maldonado,2020.000000,395.750000,252.500000,0.19900,0.1945,0.00450,0.34525,0.334250,0.011000,0.271250,0.26800,0.003250,0.318
4,456781,Donovan Solano,2020.666667,292.000000,213.666667,0.29800,0.2860,0.01200,0.41500,0.425000,-0.010000,0.329667,0.33100,-0.001333,0.397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,681082,Bryson Stott,2022.000000,466.000000,340.000000,0.23400,0.2370,-0.00300,0.35800,0.340000,0.018000,0.289000,0.28400,0.005000,0.441
243,682928,CJ Abrams,2022.000000,302.000000,238.000000,0.24600,0.2410,0.00500,0.32400,0.316000,0.008000,0.267000,0.26600,0.001000,0.411
244,682985,Riley Greene,2022.000000,418.000000,259.000000,0.25300,0.2460,0.00700,0.36200,0.388000,-0.026000,0.303000,0.31600,-0.013000,0.464
245,683734,Andrew Vaughn,2021.500000,512.000000,370.000000,0.25300,0.2490,0.00400,0.41250,0.424500,-0.012000,0.316500,0.32550,-0.009000,0.427


In [10]:
power_column_order = [
    col for col in players_power_output.columns if col not in ['avg_slg', 'slg_2023']
] + ['avg_slg', 'slg_2023']

players_power_output = players_power_output[power_column_order]

players_power_output

,player_id,name,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,avg_slg,slg_2023
0,408234,Miguel Cabrera,2020.666667,502.666667,351.333333,0.26400,0.2460,0.01800,0.395667,-0.028667,0.299333,0.30700,-0.007667,0.36700,0.332
1,444482,David Peralta,2020.000000,516.250000,365.500000,0.26950,0.2540,0.01550,0.398750,0.049750,0.333750,0.31500,0.018750,0.44850,0.399
2,453568,Charlie Blackmon,2020.000000,622.250000,457.500000,0.28475,0.2795,0.00525,0.449000,0.028000,0.351250,0.34325,0.008000,0.47700,0.460
3,455117,Martin Maldonado,2020.000000,395.750000,252.500000,0.19900,0.1945,0.00450,0.334250,0.011000,0.271250,0.26800,0.003250,0.34525,0.318
4,456781,Donovan Solano,2020.666667,292.000000,213.666667,0.29800,0.2860,0.01200,0.425000,-0.010000,0.329667,0.33100,-0.001333,0.41500,0.397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,681082,Bryson Stott,2022.000000,466.000000,340.000000,0.23400,0.2370,-0.00300,0.340000,0.018000,0.289000,0.28400,0.005000,0.35800,0.441
243,682928,CJ Abrams,2022.000000,302.000000,238.000000,0.24600,0.2410,0.00500,0.316000,0.008000,0.267000,0.26600,0.001000,0.32400,0.411
244,682985,Riley Greene,2022.000000,418.000000,259.000000,0.25300,0.2460,0.00700,0.388000,-0.026000,0.303000,0.31600,-0.013000,0.36200,0.464
245,683734,Andrew Vaughn,2021.500000,512.000000,370.000000,0.25300,0.2490,0.00400,0.424500,-0.012000,0.316500,0.32550,-0.009000,0.41250,0.427


In [11]:
avg_slg = players_power_output['avg_slg']
slg_2023 = players_power_output['slg_2023']

z_scores_avg_slg = zscore(avg_slg)
z_score_slg_2023 = zscore(slg_2023)

# Add new columns with Z-scores to the DataFrame
players_power_output['z_scores_avg_slg'] = z_scores_avg_slg

/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_27485/3794621271.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_power_output['z_scores_avg_slg'] = z_scores_avg_slg


In [12]:
players_power_output['zscore_difference'] = z_score_slg_2023 - z_scores_avg_slg

players_power_output

/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_27485/2861846325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_power_output['zscore_difference'] = z_score_slg_2023 - z_scores_avg_slg


,player_id,name,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,avg_slg,slg_2023,z_scores_avg_slg,zscore_difference
0,408234,Miguel Cabrera,2020.666667,502.666667,351.333333,0.26400,0.2460,0.01800,0.395667,-0.028667,0.299333,0.30700,-0.007667,0.36700,0.332,-1.145143,-0.182272
1,444482,David Peralta,2020.000000,516.250000,365.500000,0.26950,0.2540,0.01550,0.398750,0.049750,0.333750,0.31500,0.018750,0.44850,0.399,0.242665,-0.596952
2,453568,Charlie Blackmon,2020.000000,622.250000,457.500000,0.28475,0.2795,0.00525,0.449000,0.028000,0.351250,0.34325,0.008000,0.47700,0.460,0.727972,-0.196277
3,455117,Martin Maldonado,2020.000000,395.750000,252.500000,0.19900,0.1945,0.00450,0.334250,0.011000,0.271250,0.26800,0.003250,0.34525,0.318,-1.515509,-0.015246
4,456781,Donovan Solano,2020.666667,292.000000,213.666667,0.29800,0.2860,0.01200,0.425000,-0.010000,0.329667,0.33100,-0.001333,0.41500,0.397,-0.327784,-0.055552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,681082,Bryson Stott,2022.000000,466.000000,340.000000,0.23400,0.2370,-0.00300,0.340000,0.018000,0.289000,0.28400,0.005000,0.35800,0.441,-1.298398,1.554131
243,682928,CJ Abrams,2022.000000,302.000000,238.000000,0.24600,0.2410,0.00500,0.316000,0.008000,0.267000,0.26600,0.001000,0.32400,0.411,-1.877361,1.697365
244,682985,Riley Greene,2022.000000,418.000000,259.000000,0.25300,0.2460,0.00700,0.388000,-0.026000,0.303000,0.31600,-0.013000,0.36200,0.464,-1.230285,1.820077
245,683734,Andrew Vaughn,2021.500000,512.000000,370.000000,0.25300,0.2490,0.00400,0.424500,-0.012000,0.316500,0.32550,-0.009000,0.41250,0.427,-0.370354,0.422748


In [13]:
# Filter the DataFrame for the relevant years (2019, 2021, 2022)
relevant_years = [2018, 2019, 2021, 2022]
filtered_batting_data = batting_data[batting_data['year'].isin(relevant_years)]


bip_2023 = batting_23[['player_id', 'bip']].rename(columns={'bip': 'bip_2023'})
contact_grouped_data = filtered_batting_data.groupby(['player_id', 'name']).mean().reset_index().rename(columns={'bip': 'avg_bip'})

# Merge player names with the result DataFrame
# Merge player names and calculated results
players_contact_output = pd.merge(
    contact_grouped_data,
    bip_2023,
    on=['player_id']
)

In [14]:
contact_column_order = [
    col for col in players_contact_output.columns if col not in ['avg_bip', 'bip_2023']
] + ['avg_bip', 'bip_2023']

players_contact_output = players_contact_output[contact_column_order]

players_contact_output

,player_id,name,year,pa,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,avg_bip,bip_2023
0,408234,Miguel Cabrera,2020.666667,502.666667,0.26400,0.2460,0.01800,0.36700,0.395667,-0.028667,0.299333,0.30700,-0.007667,351.333333,195
1,444482,David Peralta,2020.000000,516.250000,0.26950,0.2540,0.01550,0.44850,0.398750,0.049750,0.333750,0.31500,0.018750,365.500000,257
2,453568,Charlie Blackmon,2020.000000,622.250000,0.28475,0.2795,0.00525,0.47700,0.449000,0.028000,0.351250,0.34325,0.008000,457.500000,207
3,455117,Martin Maldonado,2020.000000,395.750000,0.19900,0.1945,0.00450,0.34525,0.334250,0.011000,0.271250,0.26800,0.003250,252.500000,189
4,456781,Donovan Solano,2020.666667,292.000000,0.29800,0.2860,0.01200,0.41500,0.425000,-0.010000,0.329667,0.33100,-0.001333,213.666667,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,681082,Bryson Stott,2022.000000,466.000000,0.23400,0.2370,-0.00300,0.35800,0.340000,0.018000,0.289000,0.28400,0.005000,340.000000,395
243,682928,CJ Abrams,2022.000000,302.000000,0.24600,0.2410,0.00500,0.32400,0.316000,0.008000,0.267000,0.26600,0.001000,238.000000,340
244,682985,Riley Greene,2022.000000,418.000000,0.25300,0.2460,0.00700,0.36200,0.388000,-0.026000,0.303000,0.31600,-0.013000,259.000000,242
245,683734,Andrew Vaughn,2021.500000,512.000000,0.25300,0.2490,0.00400,0.41250,0.424500,-0.012000,0.316500,0.32550,-0.009000,370.000000,340


In [15]:
avg_bip = players_contact_output['avg_bip']
bip_2023 = players_contact_output['bip_2023']

z_scores_avg_bip = zscore(avg_bip)
z_score_bip_2023 = zscore(bip_2023)

# Add new columns with Z-scores to the DataFrame
players_contact_output['z_scores_avg_bip'] = z_scores_avg_bip

In [16]:
players_contact_output['zscore_difference'] = z_score_bip_2023 - z_scores_avg_bip

players_contact_output

,player_id,name,year,pa,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,avg_bip,bip_2023,z_scores_avg_bip,zscore_difference
0,408234,Miguel Cabrera,2020.666667,502.666667,0.26400,0.2460,0.01800,0.36700,0.395667,-0.028667,0.299333,0.30700,-0.007667,351.333333,195,0.199069,-1.226383
1,444482,David Peralta,2020.000000,516.250000,0.26950,0.2540,0.01550,0.44850,0.398750,0.049750,0.333750,0.31500,0.018750,365.500000,257,0.367913,-0.585828
2,453568,Charlie Blackmon,2020.000000,622.250000,0.28475,0.2795,0.00525,0.47700,0.449000,0.028000,0.351250,0.34325,0.008000,457.500000,207,1.464401,-2.335057
3,455117,Martin Maldonado,2020.000000,395.750000,0.19900,0.1945,0.00450,0.34525,0.334250,0.011000,0.271250,0.26800,0.003250,252.500000,189,-0.978861,-0.126781
4,456781,Donovan Solano,2020.666667,292.000000,0.29800,0.2860,0.01200,0.41500,0.425000,-0.010000,0.329667,0.33100,-0.001333,213.666667,223,-1.441691,0.779912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,681082,Bryson Stott,2022.000000,466.000000,0.23400,0.2370,-0.00300,0.35800,0.340000,0.018000,0.289000,0.28400,0.005000,340.000000,395,0.063995,1.519655
243,682928,CJ Abrams,2022.000000,302.000000,0.24600,0.2410,0.00500,0.32400,0.316000,0.008000,0.267000,0.26600,0.001000,238.000000,340,-1.151677,2.017312
244,682985,Riley Greene,2022.000000,418.000000,0.25300,0.2460,0.00700,0.36200,0.388000,-0.026000,0.303000,0.31600,-0.013000,259.000000,242,-0.901392,0.487655
245,683734,Andrew Vaughn,2021.500000,512.000000,0.25300,0.2490,0.00400,0.41250,0.424500,-0.012000,0.316500,0.32550,-0.009000,370.000000,340,0.421545,0.444089


In [33]:
# Filter the DataFrame for the relevant years (2019, 2021, 2022)
relevant_years = [2018, 2019, 2021, 2022]
filtered_batting_data = batting_data[batting_data['year'].isin(relevant_years)]


bip_2023 = batting_23[['player_id', 'ba']].rename(columns={'ba': 'ba_2023'})
ba_grouped_data = filtered_batting_data.groupby(['player_id', 'name']).mean().reset_index().rename(columns={'ba': 'avg_ba'})

# Merge player names with the result DataFrame
# Merge player names and calculated results
players_ba_output = pd.merge(
    ba_grouped_data,
    bip_2023,
    on=['player_id']
)

In [34]:
ba_column_order = [
    col for col in players_ba_output.columns if col not in ['avg_ba', 'ba_2023']
] + ['avg_ba', 'ba_2023']

players_ba_output = players_ba_output[ba_column_order]

players_ba_output

,player_id,name,year,pa,bip,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,avg_ba,ba_2023
0,408234,Miguel Cabrera,2020.67,502.67,351.33,0.25,0.02,0.37,0.40,-0.03,0.30,0.31,-0.01,0.26,0.25
1,444482,David Peralta,2020.00,516.25,365.50,0.25,0.02,0.45,0.40,0.05,0.33,0.32,0.02,0.27,0.27
2,453568,Charlie Blackmon,2020.00,622.25,457.50,0.28,0.01,0.48,0.45,0.03,0.35,0.34,0.01,0.28,0.28
3,455117,Martin Maldonado,2020.00,395.75,252.50,0.19,0.00,0.35,0.33,0.01,0.27,0.27,0.00,0.20,0.18
4,456781,Donovan Solano,2020.67,292.00,213.67,0.29,0.01,0.42,0.42,-0.01,0.33,0.33,-0.00,0.30,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,681082,Bryson Stott,2022.00,466.00,340.00,0.24,-0.00,0.36,0.34,0.02,0.29,0.28,0.01,0.23,0.30
243,682928,CJ Abrams,2022.00,302.00,238.00,0.24,0.01,0.32,0.32,0.01,0.27,0.27,0.00,0.25,0.25
244,682985,Riley Greene,2022.00,418.00,259.00,0.25,0.01,0.36,0.39,-0.03,0.30,0.32,-0.01,0.25,0.29
245,683734,Andrew Vaughn,2021.50,512.00,370.00,0.25,0.00,0.41,0.42,-0.01,0.32,0.33,-0.01,0.25,0.25


In [35]:
avg_ba = players_ba_output['avg_ba']
ba_2023 = players_ba_output['ba_2023']

z_scores_avg_ba = zscore(avg_ba)
z_score_ba_2023 = zscore(ba_2023)

# Add new columns with Z-scores to the DataFrame
players_ba_output['z_scores_avg_ba'] = z_scores_avg_ba
players_ba_output['zscore_difference'] = z_score_ba_2023 - z_scores_avg_ba

players_ba_output

,player_id,name,year,pa,bip,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,avg_ba,ba_2023,z_scores_avg_ba,zscore_difference
0,408234,Miguel Cabrera,2020.67,502.67,351.33,0.25,0.02,0.37,0.40,-0.03,0.30,0.31,-0.01,0.26,0.25,0.25,-0.52
1,444482,David Peralta,2020.00,516.25,365.50,0.25,0.02,0.45,0.40,0.05,0.33,0.32,0.02,0.27,0.27,0.47,0.09
2,453568,Charlie Blackmon,2020.00,622.25,457.50,0.28,0.01,0.48,0.45,0.03,0.35,0.34,0.01,0.28,0.28,1.07,-0.33
3,455117,Martin Maldonado,2020.00,395.75,252.50,0.19,0.00,0.35,0.33,0.01,0.27,0.27,0.00,0.20,0.18,-2.30,-0.30
4,456781,Donovan Solano,2020.67,292.00,213.67,0.29,0.01,0.42,0.42,-0.01,0.33,0.33,-0.00,0.30,0.28,1.59,-0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,681082,Bryson Stott,2022.00,466.00,340.00,0.24,-0.00,0.36,0.34,0.02,0.29,0.28,0.01,0.23,0.30,-0.92,2.39
243,682928,CJ Abrams,2022.00,302.00,238.00,0.24,0.01,0.32,0.32,0.01,0.27,0.27,0.00,0.25,0.25,-0.45,0.29
244,682985,Riley Greene,2022.00,418.00,259.00,0.25,0.01,0.36,0.39,-0.03,0.30,0.32,-0.01,0.25,0.29,-0.18,1.57
245,683734,Andrew Vaughn,2021.50,512.00,370.00,0.25,0.00,0.41,0.42,-0.01,0.32,0.33,-0.01,0.25,0.25,-0.18,0.12


In [38]:
players_ba_output.to_csv('Resources/ba_for_learning.csv', encoding="utf-8", index=False)

In [39]:
players_batting_output.to_csv('Resources/batting_for_learning.csv', encoding="utf-8", index=False)

In [40]:
players_power_output.to_csv('Resources/power_for_learning.csv', encoding="utf-8", index=False)

In [41]:
players_contact_output.to_csv('Resources/contact_for_learning.csv', encoding="utf-8", index=False)

## Pitchers by zERA ZScore

In [20]:
from pybaseball import statcast_pitcher_expected_stats

# get data for all qualified pitchers in 2019
pitching_18 = statcast_pitcher_expected_stats(2018)
pitching_19 = statcast_pitcher_expected_stats(2019)
pitching_21 = statcast_pitcher_expected_stats(2021)
pitching_22 = statcast_pitcher_expected_stats(2022)
pitching_23 = statcast_pitcher_expected_stats(2023)

pitcher_combined_data = pd.concat([pitching_18, pitching_19, pitching_21, pitching_22, pitching_23])
pitcher_combined_data

,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,xera,era_minus_xera_diff
0,Keuchel,Dallas,572971,2018,874,661,0.263,0.247,0.016,0.393,0.362,0.031,0.305,0.293,0.012,3.74,3.60,0.138
1,Shields,James,448306,2018,871,630,0.245,0.262,-0.017,0.435,0.466,-0.031,0.326,0.350,-0.024,4.53,5.33,-0.801
2,Scherzer,Max,453286,2018,866,503,0.188,0.185,0.003,0.332,0.316,0.016,0.252,0.248,0.004,2.53,2.56,-0.031
3,Freeland,Kyle,607536,2018,844,595,0.240,0.238,0.002,0.358,0.373,-0.015,0.292,0.300,-0.008,2.85,3.79,-0.943
4,Kluber,Corey,446372,2018,842,583,0.223,0.227,-0.004,0.367,0.369,-0.002,0.270,0.277,-0.007,2.89,3.20,-0.312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,Moore,Matt,519043,2023,160,102,0.186,0.213,-0.027,0.338,0.394,-0.056,0.255,0.295,-0.040,2.19,3.60,-1.405
372,May,Trevor,543507,2023,159,101,0.214,0.248,-0.034,0.365,0.398,-0.033,0.316,0.346,-0.030,4.37,5.12,-0.749
373,Hentges,Sam,656529,2023,159,106,0.271,0.275,-0.004,0.368,0.413,-0.045,0.314,0.334,-0.020,4.95,4.73,0.224
374,Coulombe,Danny,543056,2023,159,100,0.230,0.205,0.025,0.324,0.310,0.014,0.270,0.255,0.015,2.77,2.67,0.099


In [21]:
relevant_years = [2018, 2019, 2021, 2022]
filtered_data = pitcher_combined_data[pitcher_combined_data['year'].isin(relevant_years)]

pitching_data_2023 = pitcher_combined_data[pitcher_combined_data['year'] == 2023]
# Merge the average wOBA with the 2023 wOBA for each player
grouped_data = filtered_data.groupby(['player_id']).mean().reset_index().rename(columns={'xera': 'avg_zERA'})

pitching_data_2023

/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_27485/2014980854.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_data = filtered_data.groupby(['player_id']).mean().reset_index().rename(columns={'xera': 'avg_zERA'})


,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,xera,era_minus_xera_diff
0,Alcantara,Sandy,645261,2023,704,511,0.247,0.264,-0.017,0.389,0.413,-0.024,0.298,0.319,-0.021,4.16,4.27,-0.112
1,Webb,Logan,657277,2023,670,477,0.248,0.250,-0.002,0.385,0.399,-0.014,0.289,0.297,-0.008,3.35,3.65,-0.295
2,Gallen,Zac,668678,2023,668,452,0.229,0.249,-0.020,0.366,0.419,-0.053,0.276,0.309,-0.033,3.11,3.98,-0.873
3,Keller,Mitch,656605,2023,665,436,0.251,0.243,0.008,0.405,0.390,0.015,0.312,0.308,0.004,4.22,3.95,0.271
4,Nola,Aaron,605400,2023,656,450,0.238,0.241,-0.003,0.431,0.419,0.012,0.306,0.307,-0.001,4.49,3.92,0.571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,Moore,Matt,519043,2023,160,102,0.186,0.213,-0.027,0.338,0.394,-0.056,0.255,0.295,-0.040,2.19,3.60,-1.405
372,May,Trevor,543507,2023,159,101,0.214,0.248,-0.034,0.365,0.398,-0.033,0.316,0.346,-0.030,4.37,5.12,-0.749
373,Hentges,Sam,656529,2023,159,106,0.271,0.275,-0.004,0.368,0.413,-0.045,0.314,0.334,-0.020,4.95,4.73,0.224
374,Coulombe,Danny,543056,2023,159,100,0.230,0.205,0.025,0.324,0.310,0.014,0.270,0.255,0.015,2.77,2.67,0.099


In [22]:
zera_2023 = pitching_data_2023[['player_id', 'xera']].rename(columns={'xera': 'zERA_2023'})
zera_2023

,player_id,zERA_2023
0,645261,4.27
1,657277,3.65
2,668678,3.98
3,656605,3.95
4,605400,3.92
...,...,...
371,519043,3.60
372,543507,5.12
373,656529,4.73
374,543056,2.67


In [23]:
# Merge player names with the result DataFrame
# Merge player names and calculated results
pitcher_output = pd.merge(
    grouped_data,
    zera_2023,
    on=['player_id']
)

# Display the result
pitcher_output

,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,avg_zERA,era_minus_xera_diff,zERA_2023
0,425794,2020.666667,792.00,571.333333,0.251333,0.262000,-0.010667,0.391333,0.412000,-0.020667,0.303000,0.319667,-0.016667,3.650000,4.396667,-0.744333,7.62
1,425844,2020.000000,732.75,550.250000,0.249250,0.258250,-0.009000,0.401250,0.416500,-0.015250,0.294750,0.309750,-0.015000,3.495000,4.100000,-0.605750,5.19
2,434378,2019.666667,782.00,480.666667,0.186000,0.192333,-0.006333,0.330333,0.328667,0.001667,0.241333,0.246667,-0.005333,2.283333,2.553333,-0.268333,3.42
3,445276,2020.000000,272.50,163.750000,0.189500,0.189000,0.000500,0.336250,0.313000,0.023250,0.261000,0.256500,0.004500,3.080000,2.762500,0.317500,2.71
4,446372,2020.333333,624.00,441.000000,0.248000,0.242667,0.005333,0.389000,0.385667,0.003333,0.298667,0.298667,0.000000,3.683333,3.743333,-0.060667,6.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,678394,2022.000000,268.00,184.000000,0.315000,0.246000,0.069000,0.416000,0.347000,0.069000,0.355000,0.303000,0.052000,4.710000,3.800000,0.909000,4.09
274,680686,2021.500000,478.00,308.000000,0.238500,0.219000,0.019500,0.497500,0.436500,0.061000,0.349000,0.322000,0.027000,5.255000,4.380000,0.870500,4.95
275,680694,2022.000000,509.00,344.000000,0.265000,0.253000,0.012000,0.419000,0.404000,0.015000,0.334000,0.327000,0.007000,4.890000,4.490000,0.405000,4.01
276,681911,2022.000000,227.00,122.000000,0.187000,0.188000,-0.001000,0.247000,0.295000,-0.048000,0.244000,0.261000,-0.017000,2.150000,2.790000,-0.637000,3.63


In [24]:
pitcher_output = pitcher_output.drop(columns='year')
pd.set_option('display.float_format', '{:.2f}'.format)

In [25]:
# Assuming 'data' is your DataFrame
# Exclude non-numeric columns if needed
numeric_columns = pitcher_output.select_dtypes(include=['float64', 'int64']).columns

# Exclude 'player_id' from numeric columns
numeric_columns = numeric_columns.drop('player_id')

# Calculate z-scores for numeric columns (excluding 'player_id')
data_zscored = pitcher_output.copy()
data_zscored[numeric_columns] = zscore(pitcher_output[numeric_columns])

data_zscored['zscore_difference'] = data_zscored['zERA_2023'] - data_zscored['avg_zERA'] 

# Now 'data_zscored' contains z-scored values for numeric columns (excluding 'player_id')
data_zscored

,player_id,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,avg_zERA,era_minus_xera_diff,zERA_2023,zscore_difference
0,425794,2.14,2.30,0.58,1.15,-0.92,0.12,0.65,-0.90,0.09,0.68,-1.05,-0.18,0.68,-1.16,3.42,2.74
1,425844,1.80,2.14,0.51,1.01,-0.80,0.30,0.74,-0.70,-0.18,0.30,-0.93,-0.35,0.27,-0.92,0.98,0.71
2,434378,2.08,1.59,-1.63,-1.55,-0.60,-0.96,-1.06,-0.09,-1.95,-2.07,-0.27,-1.64,-1.87,-0.32,-0.80,1.07
3,445276,-0.84,-0.92,-1.51,-1.68,-0.11,-0.86,-1.38,0.69,-1.30,-1.70,0.41,-0.79,-1.58,0.71,-1.51,0.07
4,446372,1.18,1.27,0.46,0.40,0.24,0.08,0.11,-0.03,-0.05,-0.11,0.10,-0.15,-0.22,0.04,1.99,2.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,678394,-0.87,-0.76,2.73,0.53,4.85,0.56,-0.68,2.34,1.81,0.05,3.67,0.95,-0.15,1.76,-0.12,0.02
274,680686,0.34,0.22,0.14,-0.52,1.26,2.01,1.15,2.05,1.61,0.76,1.95,1.54,0.65,1.69,0.74,0.09
275,680694,0.52,0.50,1.04,0.80,0.72,0.61,0.48,0.39,1.12,0.95,0.58,1.15,0.81,0.87,-0.20,-1.01
276,681911,-1.10,-1.26,-1.60,-1.72,-0.22,-2.45,-1.74,-1.89,-1.86,-1.53,-1.07,-1.79,-1.54,-0.97,-0.58,0.95


In [26]:
# Display the result
column_order = [
    'player_id', 'pa', 'bip', 'ba', 'est_ba', 'est_ba_minus_ba_diff',
    'slg', 'est_slg', 'est_slg_minus_slg_diff', 'woba', 'est_woba',
    'est_woba_minus_woba_diff', 'era', 'era_minus_xera_diff',
    'zERA_2023', 'avg_zERA', 'zscore_difference'
]

# Reorder the columns
data_zscored = data_zscored[column_order]

# Display the result
data_zscored

,player_id,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,era_minus_xera_diff,zERA_2023,avg_zERA,zscore_difference
0,425794,2.14,2.30,0.58,1.15,-0.92,0.12,0.65,-0.90,0.09,0.68,-1.05,-0.18,-1.16,3.42,0.68,2.74
1,425844,1.80,2.14,0.51,1.01,-0.80,0.30,0.74,-0.70,-0.18,0.30,-0.93,-0.35,-0.92,0.98,0.27,0.71
2,434378,2.08,1.59,-1.63,-1.55,-0.60,-0.96,-1.06,-0.09,-1.95,-2.07,-0.27,-1.64,-0.32,-0.80,-1.87,1.07
3,445276,-0.84,-0.92,-1.51,-1.68,-0.11,-0.86,-1.38,0.69,-1.30,-1.70,0.41,-0.79,0.71,-1.51,-1.58,0.07
4,446372,1.18,1.27,0.46,0.40,0.24,0.08,0.11,-0.03,-0.05,-0.11,0.10,-0.15,0.04,1.99,-0.22,2.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,678394,-0.87,-0.76,2.73,0.53,4.85,0.56,-0.68,2.34,1.81,0.05,3.67,0.95,1.76,-0.12,-0.15,0.02
274,680686,0.34,0.22,0.14,-0.52,1.26,2.01,1.15,2.05,1.61,0.76,1.95,1.54,1.69,0.74,0.65,0.09
275,680694,0.52,0.50,1.04,0.80,0.72,0.61,0.48,0.39,1.12,0.95,0.58,1.15,0.87,-0.20,0.81,-1.01
276,681911,-1.10,-1.26,-1.60,-1.72,-0.22,-2.45,-1.74,-1.89,-1.86,-1.53,-1.07,-1.79,-0.97,-0.58,-1.54,0.95


In [27]:
 #Write the DataFrame to a CSV file
pitcher_output.to_csv('Resources/pitcher_zscores_for_learning.csv', index=False)

In [28]:
avg_zera = pitcher_output['avg_zERA']
zera_2023 = pitcher_output['zERA_2023']
# 
z_scores_avg_zera = zscore(avg_zera)
z_score_zera_2023 = zscore(zera_2023)

In [29]:
# Calculate zscore_difference and assign z_scores_avg_zera
pitcher_output['zscore_difference'] = z_score_zera_2023 - z_scores_avg_zera


# Display the result
column_order = [
    'player_id', 'pa', 'bip', 'ba', 'est_ba', 'est_ba_minus_ba_diff',
    'slg', 'est_slg', 'est_slg_minus_slg_diff', 'woba', 'est_woba',
    'est_woba_minus_woba_diff', 'era', 'era_minus_xera_diff',
    'zERA_2023', 'avg_zERA', 'zscore_difference'
]

# Reorder the columns
pitcher_output = pitcher_output[column_order]

# Display the result
pitcher_output

,player_id,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,era_minus_xera_diff,zERA_2023,avg_zERA,zscore_difference
0,425794,792.00,571.33,0.25,0.26,-0.01,0.39,0.41,-0.02,0.30,0.32,-0.02,3.65,-0.74,7.62,4.40,2.74
1,425844,732.75,550.25,0.25,0.26,-0.01,0.40,0.42,-0.02,0.29,0.31,-0.01,3.50,-0.61,5.19,4.10,0.71
2,434378,782.00,480.67,0.19,0.19,-0.01,0.33,0.33,0.00,0.24,0.25,-0.01,2.28,-0.27,3.42,2.55,1.07
3,445276,272.50,163.75,0.19,0.19,0.00,0.34,0.31,0.02,0.26,0.26,0.00,3.08,0.32,2.71,2.76,0.07
4,446372,624.00,441.00,0.25,0.24,0.01,0.39,0.39,0.00,0.30,0.30,0.00,3.68,-0.06,6.19,3.74,2.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,678394,268.00,184.00,0.32,0.25,0.07,0.42,0.35,0.07,0.35,0.30,0.05,4.71,0.91,4.09,3.80,0.02
274,680686,478.00,308.00,0.24,0.22,0.02,0.50,0.44,0.06,0.35,0.32,0.03,5.26,0.87,4.95,4.38,0.09
275,680694,509.00,344.00,0.27,0.25,0.01,0.42,0.40,0.01,0.33,0.33,0.01,4.89,0.41,4.01,4.49,-1.01
276,681911,227.00,122.00,0.19,0.19,-0.00,0.25,0.29,-0.05,0.24,0.26,-0.02,2.15,-0.64,3.63,2.79,0.95


In [30]:
 #Write the DataFrame to a CSV file
pitcher_output.to_csv('Resources/pitcher_output_for_learning.csv', index=False)